https://adventofcode.com/2023/day/23

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from scipy.spatial import distance_matrix

input_test = r"""#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#"""

with open('data/day_23.txt') as txtfile:
    input_day = txtfile.read()

def get_data(prod=0):
    input_data = input_day if prod else input_test
    return input_data


In [7]:
data = get_data(1)
df = pd.DataFrame([list(line) for line in data.splitlines()])
df

,0,1,2,3,4,5,6,7,8,9,...,131,132,133,134,135,136,137,138,139,140
0,#,.,#,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
1,#,.,.,.,#,.,.,.,#,.,...,.,.,.,.,.,#,.,.,.,#
2,#,#,#,.,#,.,#,.,#,.,...,#,#,#,#,.,#,.,#,.,#
3,#,.,.,.,#,.,#,.,#,.,...,.,.,.,.,.,#,.,#,.,#
4,#,.,#,#,#,.,#,.,#,#,...,.,#,#,#,#,#,.,#,.,#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,#,#,#,#,#,.,#,.,#,.,...,#,#,.,#,#,#,#,#,.,#
137,#,.,.,.,.,.,#,.,#,.,...,.,#,.,#,.,.,.,.,.,#
138,#,.,#,#,#,#,#,.,#,.,...,.,#,.,#,.,#,#,#,#,#
139,#,.,.,.,.,.,.,.,#,.,...,.,.,.,#,.,.,.,.,.,#


In [8]:
cells = np.array([[i, c] for i in df.index for c in df.columns])
dist = distance_matrix(cells, cells, p=1)
neighs = {}
for i in range(len(cells)):
    cell = tuple(cells[i])
    if df.loc[cell]!="#":
        neighs[cell] = [tuple(i) for i in cells[(dist[i] == 1)] if df.loc[tuple(i)]!="#"]

edges = [(k, n) for k, v in neighs.items() for n in v]
G = nx.Graph()
G.add_edges_from(edges)

In [15]:
paths = nx.all_simple_paths(G, source=(0,1), target=(df.shape[0]-1, df.shape[1]-2))
final_paths = []
for pos_p, p in enumerate(paths):
    if (pos_p % 20)==0: print(pos_p)
    errors = 0
    for pos, node in enumerate(p):
        if pos > 0:
            v = node[0] - p[pos-1][0]
            h = node[1] - p[pos-1][1]
            if h==0:
                if v > 0:
                    if df.loc[node] == "^":
                        #print("Not valid ^", node)
                        errors += 1
                        break
                else:
                    if df.loc[node] == "v":
                        #print("Not valid v", node)
                        errors += 1
                        break
            elif h > 0:
                if df.loc[node] == "<":
                    #print("Not valid <", node)
                    errors += 1
                    break
            else:
                if df.loc[node] == ">":
                    #print("Not valid >", node)
                    errors += 1
                    break
    if errors==0:
        final_paths.append(p)
        
final_paths

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080


KeyboardInterrupt: 

In [16]:
max([len(paths[p]) for p in final_paths]) - 1

ValueError: max() arg is an empty sequence

In [80]:
paths = list(nx.all_simple_paths(G, source=(0,1), target=(22,21)))
p = 0
df_copy = df.copy()
for i in paths[0]:
    if df_copy.loc[i] not in "<>v^":
        df_copy.loc[i] = "O"
df_copy
    

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,#,O,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#,#
1,#,O,O,O,O,O,O,O,#,#,#,#,#,#,#,#,#,O,O,O,#,#,#
2,#,#,#,#,#,#,#,O,#,#,#,#,#,#,#,#,#,O,#,O,#,#,#
3,#,#,#,O,O,O,O,O,#,O,>,O,>,O,#,#,#,O,#,O,#,#,#
4,#,#,#,v,#,#,#,#,#,O,#,v,#,O,#,#,#,O,#,O,#,#,#
5,#,#,#,O,>,O,O,O,#,O,#,.,#,O,O,O,O,O,#,O,O,O,#
6,#,#,#,v,#,#,#,O,#,O,#,.,#,#,#,#,#,#,#,#,#,O,#
7,#,#,#,.,.,.,#,O,#,O,#,.,.,.,.,.,.,.,#,O,O,O,#
8,#,#,#,#,#,.,#,O,#,O,#,#,#,#,#,#,#,.,#,O,#,#,#
9,#,.,.,.,.,.,#,O,#,O,#,.,.,.,.,.,.,.,#,O,O,O,#


In [23]:
pd.options.display.max_columns = 50